In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
data = pd.read_csv(r"C:\Users\dell\Desktop\MyDocs\Docs\MK\geophone-sensor-data.csv")

In [5]:
features = data[['mean', 'top_3_mean', 'min', 'max', 'std_dev', 'median', 'q1', 'q3', 'skewness', 'dominant_freq', 'energy']]
labels = data['name']  
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)  
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
def create_sequences(data, labels, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(labels[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 10
X, y = create_sequences(features_scaled, labels_encoded, time_steps)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax'))  
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

C:\Users\dell\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.4347 - loss: 1.2512 - val_accuracy: 0.6294 - val_loss: 0.8126
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6103 - loss: 0.8511 - val_accuracy: 0.6294 - val_loss: 0.7599
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6023 - loss: 0.7915 - val_accuracy: 0.6364 - val_loss: 0.7425
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6099 - loss: 0.7953 - val_accuracy: 0.6503 - val_loss: 0.7290
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6339 - loss: 0.7465 - val_accuracy: 0.6608 - val_loss: 0.7153
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6263 - loss: 0.7439 - val_accuracy: 0.6503 - val_loss: 0.7115
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6525 - loss: 0.7136 - val_accuracy: 0.6923 - val_loss: 0.6880
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6378 - loss: 0.7196 - val_accuracy: 0.6783 - val

In [15]:
from sklearn.metrics import classification_report, confusion_matrix

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy Score: {test_accuracy}")

y_pred = np.argmax(model.predict(X_test), axis=1)
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8966 - loss: 0.2820 
Test Accuracy Score: 0.9020978808403015
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Classification Report:
               precision    recall  f1-score   support

      Alihan       1.00      0.97      0.99        80
        Enes       0.81      0.90      0.86        73
      Furkan       0.89      0.89      0.89        76
       Yusuf       0.90      0.81      0.85        57

    accuracy                           0.90       286
   macro avg       0.90      0.90      0.90       286
weighted avg       0.91      0.90      0.90       286

Confusion Matrix:
 [[78  0  2  0]
 [ 0 66  4  3]
 [ 0  6 68  2]
 [ 0  9  2 46]]


In [21]:
new_data = features_scaled[:time_steps] 
new_data = np.expand_dims(new_data, axis=0)  
prediction = model.predict(new_data)
predicted_name = label_encoder.inverse_transform([np.argmax(prediction)])
print(f"Predicted Name is: {predicted_name[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Predicted Name is: Yusuf


In [19]:
train_loss, train_accuracy = model.evaluate(X_train, y_train)
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9218 - loss: 0.2081
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8966 - loss: 0.2820 
Train Accuracy: 0.9170
Test Accuracy: 0.9021
